In [11]:
import os
import time
import pandas as pd
import numpy as np
import networkx as nx
import collections
from scipy import sparse as sp
from scipy.stats import rankdata

import itertools
from itertools import combinations, combinations_with_replacement, cycle
from functools import reduce

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from util import *

import colorcet as cc

import bokeh
from bokeh.io import output_notebook, output_file, show, save
from bokeh.plotting import figure
from bokeh.models import (Rect, MultiLine, Circle, Span, Label,
                          GraphRenderer, StaticLayoutProvider,
                          NodesAndLinkedEdges,
                          HoverTool, TapTool, ColumnDataSource,
                          LinearColorMapper, LogColorMapper, CategoricalColorMapper,
                          CategoricalMarkerMapper,
                          BoxSelectTool,
                          ColorBar, BasicTicker, BoxZoomTool, FactorRange,
                          Range1d)
from bokeh.models import CategoricalTicker, FixedTicker, BoxAnnotation
from bokeh.models import Arrow, NormalHead, OpenHead, VeeHead, LabelSet

from bokeh.transform import transform, factor_cmap, linear_cmap, log_cmap
from bokeh.layouts import row, column, gridplot
output_notebook()

Loading BokehJS ...

In [12]:
def joint_marginal(df, c1, c2, include_fraction=False):
    """Given a dataframe and two columns, return a dataframe with the joint and marginal counts."""
    j = df.value_counts([c1, c2])
    j.name = "joint_count"
    j = j.reset_index()

    m1 = df.value_counts(c1)
    m1.name = f"{c1}_count"
    j = j.merge(m1, left_on=c1, right_index=True)

    m2 = df.value_counts(c2)
    m2.name = f"{c2}_count"
    j = j.merge(m2, left_on=c2, right_index=True)

    if include_fraction:
        j["joint_fraction"] = j["joint_count"] / j["joint_count"].sum()
        j[f"{c1}_fraction"] = j["joint_count"] / j[f"{c1}_count"]
        j[f"{c2}_fraction"] = j["joint_count"] / j[f"{c2}_count"]
    return j


In [13]:
hemibrain_version = "v1.2.1"
log_msg("Hemibrain data set being used:", hemibrain_version)

preproc_dir = "oviIN/preprocessed-" + hemibrain_version
preproc_nodes = "preprocessed_nodes.csv"
preproc_centroids = "x"
preproc_edges = "preprocessed_undirected_edges.csv"

hemibrain_dir = "oviIN_june29/clustering_" + hemibrain_version
hemibrain_nodes = "inputsoutputs_key.txt"
hemibrain_edges = "inputsoutputs.txt"
#hemibrain_nodes = "only_inputs_key.txt"
#hemibrain_edges = "only_inputs.txt"
#hemibrain_nodes = "only_outputs_key.txt"
#hemibrain_edges = "only_outputs.txt"

figure_dir = os.path.join("figures","paper")
movie_dir = os.path.join("movies")
analysis_dir = os.path.join("analysis",hemibrain_version)
obj_dir = os.path.join("obj",hemibrain_version)  # 3d objects from, e.g. fetch_roi_mesh
skel_dir = os.path.join("skeleton", hemibrain_version)  # skeleta of neurons in .csv format


for d in [figure_dir, analysis_dir, obj_dir, movie_dir]:
    if not os.path.isdir(d):
        log_msg("Creating directory", d)
        os.makedirs(d)

reneel_params = list(sorted(['0.05','0.1','0.5','0.75'], key=float))
type_params = ['celltype','instance']
list_of_params = reneel_params + type_params

log_msg("Set up directory info and useful lists")

2024 08 20 16:28:42  Hemibrain data set being used: v1.2.1
2024 08 20 16:28:42  Set up directory info and useful lists


In [14]:
from neuprint import Client
from neuprint import fetch_roi_hierarchy, fetch_neurons, NeuronCriteria as NC


auth_token_file = open("flybrain.auth.txt", 'r')
auth_token = next(auth_token_file).strip()
try:
    np_client = Client('neuprint.janelia.org', dataset='hemibrain:' + hemibrain_version, token=auth_token)
    log_msg("neuprint Client set up as `np_Client`, version", np_client.fetch_version())
except:
    np_client = None
    log_msg("neuprint Client set up failed!")

log_msg("Loading node dataframe")
if os.path.isfile(os.path.join(preproc_dir, preproc_centroids)):
    log_msg("  (with centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_centroids), index_col=0)
else:
    log_msg("  (without centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_nodes), index_col=0)
log_msg("Adding 'type group'")
HB_node_df["type_group"] = HB_node_df["celltype"]
log_msg("Done!")

log_msg("Loading directed edges from csv")
HB_edge_df = pd.read_csv(os.path.join(hemibrain_dir, hemibrain_edges), delimiter=' ', header=None).rename(columns={0: "pre", 1:"post"})
log_msg("Done!")

log_msg("Merging in cell info to edge df")
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='pre', right_index=True)
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='post', right_index=True, suffixes=['pre', 'post'])
log_msg("Done!")
pd.set_option('display.max_rows', 200)

2024 08 20 16:28:48  neuprint Client set up as `np_Client`, version 0.1.0
2024 08 20 16:28:48  Loading node dataframe
2024 08 20 16:28:48    (without centroids)
2024 08 20 16:28:48  Adding 'type group'
2024 08 20 16:28:48  Done!
2024 08 20 16:28:48  Loading directed edges from csv
2024 08 20 16:28:48  Done!
2024 08 20 16:28:48  Merging in cell info to edge df
2024 08 20 16:28:48  Done!


In [6]:
#HB_node_df

In [7]:
from neuprint import fetch_simple_connections
nc = NC(type='SLP316')
neuron_df = fetch_simple_connections("oviIN", nc)
#neuron_df

In [8]:
HB_node_df[HB_node_df['celltype']=='aMe24']

,key,0.0,0.05,0.1,0.5,0.75,1.0,instance,celltype,pre,...,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois,type_group
id,,,,,,,,,,,,,,,,,,,,,
574965048,1838,4,7,6,330,573,631,aMe24(PDM24)_L,aMe24,446,...,True,Leaves,NaN,NaN,NaN,"{'SNP(L)': {'pre': 439, 'post': 873, 'downstre...",NaN,"['ATL(L)', 'ICL(L)', 'INP', 'SCL(L)', 'SMP(L)'...","['ATL(L)', 'INP', 'SCL(L)', 'SMP(L)', 'SNP(L)']",aMe24
5813011660,2059,1,1,2,419,549,735,aMe24_R,aMe24,1346,...,False,Roughly traced,PDM24,355.0,"[17120, 17799, 6112]","{'SNP(R)': {'pre': 706, 'post': 1823, 'downstr...",NaN,"['AME(R)', 'ATL(R)', 'AVLP(R)', 'ICL(R)', 'INP...","['AME(R)', 'AVLP(R)', 'ICL(R)', 'INP', 'LH(R)'...",aMe24


In [9]:
s=HB_node_df[HB_node_df['0.0']==4]
x= s[s['0.05']==7]
f=x[x['0.1']==6]
#f[f['0.5']==330]
#z=h[h['0.75']==573]
#z[z['1.0']==631]

In [10]:
chi1 = '0.0'
fs=[]
for chi in reneel_params:
   jm = joint_marginal(HB_node_df, chi1, chi, include_fraction=True)
   display(jm)
   print(f"Clusters found at chi = {chi1}:", jm[chi1].max())
   print(f"Clusters found at chi = {chi}:", jm[chi].max())

   # sort the clusters on the y axis to get a more "diagonal" plot
   yrange = jm.sort_values([f"{chi}_fraction"], ascending=False).groupby(chi).agg({chi1: "first", f"{chi}_fraction": "first", "joint_count": "first"}).sort_values([chi1, "joint_count"], ascending=[True, False]).index
   f = figure(title=f"Clusters at chi = {chi} vs. clusters at chi = {chi1}",
            x_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.0"].max())]),
         #    y_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.05"].max())]),
            y_range=FactorRange(factors=[str(y) for y in yrange]),
            width=600, height=1000)


   jm["x"] = jm[chi1].apply(str)  # bokeh factor range has to have strings, so we have to convert these
   jm["y"] = jm[chi].apply(str)

   #fig_kws = dict(title=None,border_fill_color=None,outline_line_color=None, background_fill_color=None,)
   #f = figure(**fig_kws)

   f.rect(x="x", y="y",
         width=f"{chi1}_fraction", height=f"{chi}_fraction",
         source=jm)
   f.add_tools(HoverTool(tooltips={"Neurons": "@joint_count (@joint_fraction{%%} of Hemibrain)",
                                 f"Fraction of {chi1}": f"@{{{chi1}_fraction}}{{2.%%}}",
                                 f"Fraction of {chi}": f"@{{{chi}_fraction}}{{2.%%}}"}))
   fs.append(f)

g = gridplot(fs, ncols=2)
show(g)

,0.0,0.05,joint_count,0.0_count,0.05_count,joint_fraction,0.0_fraction,0.05_fraction
0,4,7,730,861,766,0.160475,0.847851,0.953003
24,1,7,5,1178,766,0.001099,0.004244,0.006527
25,2,7,5,782,766,0.001099,0.006394,0.006527
75,6,7,1,611,766,0.000220,0.001637,0.001305
13,5,7,23,738,766,0.005056,0.031165,0.030026
32,3,7,2,379,766,0.000440,0.005277,0.002611
9,4,4,82,861,459,0.018026,0.095238,0.178649
7,1,4,181,1178,459,0.039789,0.153650,0.394336
10,2,4,69,782,459,0.015168,0.088235,0.150327
12,6,4,53,611,459,0.011651,0.086743,0.115468


Clusters found at chi = 0.0: 6
Clusters found at chi = 0.05: 147


,0.0,0.1,joint_count,0.0_count,0.1_count,joint_fraction,0.0_fraction,0.1_fraction
0,4,6,679,861,721,0.149264,0.788618,0.941748
21,1,6,13,1178,721,0.002858,0.011036,0.018031
15,2,6,26,782,721,0.005716,0.033248,0.036061
103,6,6,1,611,721,0.000220,0.001637,0.001387
59,5,6,2,738,721,0.000440,0.002710,0.002774
...,...,...,...,...,...,...,...,...
186,5,21,1,738,1,0.000220,0.001355,1.000000
73,3,80,2,379,2,0.000440,0.005277,1.000000
115,3,24,1,379,1,0.000220,0.002639,1.000000
116,3,36,1,379,1,0.000220,0.002639,1.000000


Clusters found at chi = 0.0: 6
Clusters found at chi = 0.1: 263


,0.0,0.5,joint_count,0.0_count,0.5_count,joint_fraction,0.0_fraction,0.5_fraction
0,3,9,145,379,145,0.031875,0.382586,1.000000
4,3,13,66,379,69,0.014509,0.174142,0.956522
328,5,13,3,738,69,0.000659,0.004065,0.043478
16,3,18,38,379,56,0.008353,0.100264,0.678571
263,4,18,4,861,56,0.000879,0.004646,0.071429
...,...,...,...,...,...,...,...,...
871,6,919,1,611,1,0.000220,0.001637,1.000000
876,6,849,1,611,1,0.000220,0.001637,1.000000
882,6,916,1,611,1,0.000220,0.001637,1.000000
904,6,925,1,611,1,0.000220,0.001637,1.000000


Clusters found at chi = 0.0: 6
Clusters found at chi = 0.5: 1059


,0.0,0.75,joint_count,0.0_count,0.75_count,joint_fraction,0.0_fraction,0.75_fraction
0,3,12,80,379,95,0.017586,0.211082,0.842105
1670,4,12,1,861,95,0.000220,0.001161,0.010526
223,5,12,4,738,95,0.000879,0.005420,0.042105
41,6,12,10,611,95,0.002198,0.016367,0.105263
1,3,9,61,379,61,0.013410,0.160950,1.000000
...,...,...,...,...,...,...,...,...
1255,6,431,1,611,1,0.000220,0.001637,1.000000
1274,6,196,1,611,1,0.000220,0.001637,1.000000
1275,6,173,1,611,1,0.000220,0.001637,1.000000
1278,6,150,1,611,1,0.000220,0.001637,1.000000


Clusters found at chi = 0.0: 6
Clusters found at chi = 0.75: 1501


In [11]:
r= HB_node_df[HB_node_df['0.0']==3]
f = r[r['0.05']==5]
z= f[f['0.1']==5]
#z[z['0.5']==9]
#h[h['0.75']==496]


In [5]:
r= HB_node_df[HB_node_df['0.0']==3]
f = r[r['0.05']==5]
z= f[f['0.1']==5]
h = z[z['0.5']==9]
x=h[h['0.75']==9]
#x[x['1.0']==14]

In [13]:
output_file("/Users/rhessa/flybrain-clustering/Figures-oviIN/joint_marginal_comparison.html")
save(g, title='Joint Marginal Analysis (full)')

'/Users/rhessa/flybrain-clustering/Figures-oviIN/joint_marginal_comparison.html'

In [15]:
# Inputs
hemibrain_version = "v1.2.1"
log_msg("Hemibrain data set being used:", hemibrain_version)

preproc_dir = "oviIN/preprocessed_inputs-" + hemibrain_version
preproc_nodes = "preprocessed_nodes.csv"
preproc_centroids = "x"
preproc_edges = "preprocessed_undirected_edges.csv"

#hemibrain_dir = "oviIN/clustering_inputs_" + hemibrain_version
hemibrain_nodes = "only_inputs_key.txt"
hemibrain_edges = "inputr.txt"

figure_dir = os.path.join("figures","paper")
movie_dir = os.path.join("movies")
analysis_dir = os.path.join("analysis",hemibrain_version)
obj_dir = os.path.join("obj",hemibrain_version)  # 3d objects from, e.g. fetch_roi_mesh
skel_dir = os.path.join("skeleton", hemibrain_version)  # skeleta of neurons in .csv format


for d in [figure_dir, analysis_dir, obj_dir, movie_dir]:
    if not os.path.isdir(d):
        log_msg("Creating directory", d)
        os.makedirs(d)

reneel_params = list(sorted(['0.05','0.1','0.5','0.75'], key=float))
type_params = ['celltype','instance']
list_of_params = reneel_params + type_params

log_msg("Set up directory info and useful lists")

auth_token_file = open("flybrain.auth.txt", 'r')
auth_token = next(auth_token_file).strip()
try:
    np_client = Client('neuprint.janelia.org', dataset='hemibrain:' + hemibrain_version, token=auth_token)
    log_msg("neuprint Client set up as `np_Client`, version", np_client.fetch_version())
except:
    np_client = None
    log_msg("neuprint Client set up failed!")

log_msg("Loading node dataframe")
if os.path.isfile(os.path.join(preproc_dir, preproc_centroids)):
    log_msg("  (with centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_centroids), index_col=0)
else:
    log_msg("  (without centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_nodes), index_col=0)
log_msg("Adding 'type group'")
HB_node_df["type_group"] = HB_node_df["celltype"]
log_msg("Done!")

log_msg("Loading directed edges from csv")
HB_edge_df = pd.read_csv(hemibrain_edges, delimiter=',', header=None).rename(columns={0: "pre", 1:"post"})
log_msg("Done!")

log_msg("Merging in cell info to edge df")
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='pre', right_index=True)
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='post', right_index=True, suffixes=['pre', 'post'])
log_msg("Done!")

chi1 = '0.0'
fs=[]
for chi in reneel_params:
   jm = joint_marginal(HB_node_df, chi1, chi, include_fraction=True)
   display(jm)
   print(f"Clusters found at chi = {chi1}:", jm[chi1].max())
   print(f"Clusters found at chi = {chi}:", jm[chi].max())

   # sort the clusters on the y axis to get a more "diagonal" plot
   yrange = jm.sort_values([f"{chi}_fraction"], ascending=False).groupby(chi).agg({chi1: "first", f"{chi}_fraction": "first", "joint_count": "first"}).sort_values([chi1, "joint_count"], ascending=[True, False]).index
   f = figure(title=f"Clusters at chi = {chi} vs. clusters at chi = {chi1}",
            x_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.0"].max())]),
         #    y_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.05"].max())]),
            y_range=FactorRange(factors=[str(y) for y in yrange]),
            width=600, height=1000)


   jm["x"] = jm[chi1].apply(str)  # bokeh factor range has to have strings, so we have to convert these
   jm["y"] = jm[chi].apply(str)

   #fig_kws = dict(title=None,border_fill_color=None,outline_line_color=None, background_fill_color=None,)
   #f = figure(**fig_kws)

   f.rect(x="x", y="y",
         width=f"{chi1}_fraction", height=f"{chi}_fraction",
         source=jm)
   f.add_tools(HoverTool(tooltips={"Neurons": "@joint_count (@joint_fraction{%%} of Hemibrain)",
                                 f"Fraction of {chi1}": f"@{{{chi1}_fraction}}{{2.%%}}",
                                 f"Fraction of {chi}": f"@{{{chi}_fraction}}{{2.%%}}"}))
   fs.append(f)

k = gridplot(fs, ncols=2)

2024 08 20 16:29:03  Hemibrain data set being used: v1.2.1
2024 08 20 16:29:03  Set up directory info and useful lists
2024 08 20 16:29:03  neuprint Client set up as `np_Client`, version 0.1.0
2024 08 20 16:29:03  Loading node dataframe
2024 08 20 16:29:03    (without centroids)
2024 08 20 16:29:03  Adding 'type group'
2024 08 20 16:29:03  Done!
2024 08 20 16:29:03  Loading directed edges from csv
2024 08 20 16:29:03  Done!
2024 08 20 16:29:03  Merging in cell info to edge df
2024 08 20 16:29:03  Done!


,0.0,0.05,joint_count,0.0_count,0.05_count,joint_fraction,0.0_fraction,0.05_fraction
0,1,13,374,519,392,0.148945,0.720617,0.954082
16,5,13,13,387,392,0.005177,0.033592,0.033163
31,4,13,3,302,392,0.001195,0.009934,0.007653
41,2,13,2,716,392,0.000796,0.002793,0.005102
9,1,1,57,519,271,0.022700,0.109827,0.210332
28,4,1,4,302,271,0.001593,0.013245,0.014760
4,2,1,210,716,271,0.083632,0.293296,0.774908
10,1,2,28,519,326,0.011151,0.053950,0.085890
34,5,2,3,387,326,0.001195,0.007752,0.009202
2,3,2,293,587,326,0.116687,0.499148,0.898773


Clusters found at chi = 0.0: 5
Clusters found at chi = 0.05: 101


,0.0,0.1,joint_count,0.0_count,0.1_count,joint_fraction,0.0_fraction,0.1_fraction
0,5,20,283,387,288,0.112704,0.731266,0.982639
31,3,20,5,587,288,0.001991,0.008518,0.017361
14,5,8,27,387,221,0.010753,0.069767,0.122172
33,4,8,4,302,221,0.001593,0.013245,0.018100
101,1,8,1,519,221,0.000398,0.001927,0.004525
23,2,8,12,716,221,0.004779,0.016760,0.054299
6,3,8,177,587,221,0.070490,0.301533,0.800905
16,5,6,24,387,281,0.009558,0.062016,0.085409
47,4,6,2,302,281,0.000796,0.006623,0.007117
3,1,6,195,519,281,0.077658,0.375723,0.693950


Clusters found at chi = 0.0: 5
Clusters found at chi = 0.1: 89


,0.0,0.5,joint_count,0.0_count,0.5_count,joint_fraction,0.0_fraction,0.5_fraction
0,3,5,150,587,151,0.059737,0.255537,0.993377
507,2,5,1,716,151,0.000398,0.001397,0.006623
2,3,10,46,587,46,0.018319,0.078365,1.000000
18,3,19,19,587,19,0.007567,0.032368,1.000000
19,3,263,18,587,18,0.007168,0.030664,1.000000
...,...,...,...,...,...,...,...,...
465,4,458,1,302,1,0.000398,0.003311,1.000000
467,4,462,1,302,1,0.000398,0.003311,1.000000
477,4,466,1,302,1,0.000398,0.003311,1.000000
479,4,480,1,302,1,0.000398,0.003311,1.000000


Clusters found at chi = 0.0: 5
Clusters found at chi = 0.5: 533


,0.0,0.75,joint_count,0.0_count,0.75_count,joint_fraction,0.0_fraction,0.75_fraction
0,3,5,129,587,131,0.051374,0.219761,0.984733
493,2,5,2,716,131,0.000796,0.002793,0.015267
3,3,10,24,587,24,0.009558,0.040886,1.000000
8,3,309,14,587,14,0.005575,0.023850,1.000000
17,3,19,10,587,10,0.003982,0.017036,1.000000
...,...,...,...,...,...,...,...,...
720,4,717,1,302,1,0.000398,0.003311,1.000000
721,4,716,1,302,1,0.000398,0.003311,1.000000
748,4,225,1,302,1,0.000398,0.003311,1.000000
920,4,279,1,302,1,0.000398,0.003311,1.000000


Clusters found at chi = 0.0: 5
Clusters found at chi = 0.75: 780


In [15]:
show(k)

NameError: name 'k' is not defined

In [6]:
r=HB_node_df[HB_node_df['0.0']==3]
f= r[r['0.05']==2]
z=f[f['0.1']==2]
h=z[z['0.5']==5]
x=h[h['0.75']==5]
#x[x['1.0']==5]
# Either 5, 8 9

In [17]:
output_file("/Users/rhessa/flybrain-clustering/Figures-oviIN/joint_marginal_comparison_inputs.html")
save(k, title='Joint Marginal Analysis (Inputs)')

NameError: name 'k' is not defined

In [5]:
# outputs
hemibrain_version = "v1.2.1"
log_msg("Hemibrain data set being used:", hemibrain_version)

preproc_dir = "oviIN/preprocessed_outputs-" + hemibrain_version
preproc_nodes = "preprocessed_nodes.csv"
preproc_centroids = "x"
preproc_edges = "preprocessed_undirected_edges.csv"

hemibrain_dir = "oviIN/clustering_outputs_" + hemibrain_version
hemibrain_nodes = "only_outputs_key.txt"
hemibrain_edges = "only_outputs.txt"

figure_dir = os.path.join("figures","paper")
movie_dir = os.path.join("movies")
analysis_dir = os.path.join("analysis",hemibrain_version)
obj_dir = os.path.join("obj",hemibrain_version)  # 3d objects from, e.g. fetch_roi_mesh
skel_dir = os.path.join("skeleton", hemibrain_version)  # skeleta of neurons in .csv format


for d in [figure_dir, analysis_dir, obj_dir, movie_dir]:
    if not os.path.isdir(d):
        log_msg("Creating directory", d)
        os.makedirs(d)

reneel_params = list(sorted(['0.05','0.1','0.5','0.75'], key=float))
type_params = ['celltype','instance']
list_of_params = reneel_params + type_params

log_msg("Set up directory info and useful lists")

auth_token_file = open("flybrain.auth.txt", 'r')
auth_token = next(auth_token_file).strip()
try:
    np_client = Client('neuprint.janelia.org', dataset='hemibrain:' + hemibrain_version, token=auth_token)
    log_msg("neuprint Client set up as `np_Client`, version", np_client.fetch_version())
except:
    np_client = None
    log_msg("neuprint Client set up failed!")

log_msg("Loading node dataframe")
if os.path.isfile(os.path.join(preproc_dir, preproc_centroids)):
    log_msg("  (with centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_centroids), index_col=0)
else:
    log_msg("  (without centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_nodes), index_col=0)
log_msg("Adding 'type group'")
HB_node_df["type_group"] = HB_node_df["celltype"]
log_msg("Done!")

log_msg("Loading directed edges from csv")
HB_edge_df = pd.read_csv(os.path.join(hemibrain_dir, hemibrain_edges), delimiter=' ', header=None).rename(columns={0: "pre", 1:"post"})
log_msg("Done!")

log_msg("Merging in cell info to edge df")
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='pre', right_index=True)
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='post', right_index=True, suffixes=['pre', 'post'])
log_msg("Done!")

chi1 = '0.0'
fs=[]
for chi in reneel_params:
   jm = joint_marginal(HB_node_df, chi1, chi, include_fraction=True)
   display(jm)
   print(f"Clusters found at chi = {chi1}:", jm[chi1].max())
   print(f"Clusters found at chi = {chi}:", jm[chi].max())

   # sort the clusters on the y axis to get a more "diagonal" plot
   yrange = jm.sort_values([f"{chi}_fraction"], ascending=False).groupby(chi).agg({chi1: "first", f"{chi}_fraction": "first", "joint_count": "first"}).sort_values([chi1, "joint_count"], ascending=[True, False]).index
   f = figure(title=f"Clusters at chi = {chi} vs. clusters at chi = {chi1}",
            x_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.0"].max())]),
         #    y_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.05"].max())]),
            y_range=FactorRange(factors=[str(y) for y in yrange]),
            width=600, height=1000)


   jm["x"] = jm[chi1].apply(str)  # bokeh factor range has to have strings, so we have to convert these
   jm["y"] = jm[chi].apply(str)

   #fig_kws = dict(title=None,border_fill_color=None,outline_line_color=None, background_fill_color=None,)
   #f = figure(**fig_kws)

   f.rect(x="x", y="y",
         width=f"{chi1}_fraction", height=f"{chi}_fraction",
         source=jm)
   f.add_tools(HoverTool(tooltips={"Neurons": "@joint_count (@joint_fraction{%%} of Hemibrain)",
                                 f"Fraction of {chi1}": f"@{{{chi1}_fraction}}{{2.%%}}",
                                 f"Fraction of {chi}": f"@{{{chi}_fraction}}{{2.%%}}"}))
   fs.append(f)

i = gridplot(fs, ncols=2)

2024 08 20 16:27:58  Hemibrain data set being used: v1.2.1
2024 08 20 16:27:58  Set up directory info and useful lists
2024 08 20 16:27:58  neuprint Client set up as `np_Client`, version 0.1.0
2024 08 20 16:27:58  Loading node dataframe
2024 08 20 16:27:58    (without centroids)
2024 08 20 16:27:58  Adding 'type group'
2024 08 20 16:27:58  Done!
2024 08 20 16:27:58  Loading directed edges from csv


FileNotFoundError: [Errno 2] No such file or directory: 'oviIN/clustering_outputs_v1.2.1/only_outputs.txt'

In [20]:
output_file("/Users/rhessa/flybrain-clustering/Figures-oviIN/joint_marginal_comparison_outputs.html")
save(i, title='Join Marginal Analysis (Outputs)')

'/Users/rhessa/flybrain-clustering/Figures-oviIN/joint_marginal_comparison_outputs.html'

In [21]:
HB_node_df[HB_node_df['celltype']=='SMP550']

,key,0.0,0.05,0.1,0.5,0.75,1.0,instance,celltype,pre,...,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois,type_group
id,,,,,,,,,,,,,,,,,,,,,
452689494,789,6,32,5,234,437,498,SMP550_R,SMP550,1414,...,False,Roughly traced,PDM29,347.0,"[19511, 18367, 6016]","{'SNP(R)': {'pre': 1248, 'post': 3275, 'downst...",pSP7 candidates,"['AVLP(R)', 'INP', 'LH(R)', 'PLP(R)', 'SCL(R)'...","['AVLP(R)', 'INP', 'LH(R)', 'PLP(R)', 'SCL(R)'...",SMP550
579700707,1306,4,5,5,335,478,551,SMP550(PDM29)_L,SMP550,992,...,True,Leaves,NaN,NaN,NaN,"{'SNP(L)': {'pre': 981, 'post': 1089, 'downstr...",pSP7 candidates,"['SMP(L)', 'SMP(R)', 'SNP(L)', 'SNP(R)']","['SMP(L)', 'SMP(R)', 'SNP(L)', 'SNP(R)']",SMP550


In [7]:
r=HB_node_df[HB_node_df['0.0']==6]
f= r[r['0.05']==9]
z=f[f['0.1']==16]
#z[z['0.5']==22]
#h[h['0.75']==129]


In [23]:
HB_node_df[HB_node_df['celltype']=='pC1e']

,key,0.0,0.05,0.1,0.5,0.75,1.0,instance,celltype,pre,...,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois,type_group
id,,,,,,,,,,,,,,,,,,,,,
514850616,974,1,1,1,312,494,578,pC1e_R,pC1e,697,...,False,Traced,PDM09,356.0,"[23338, 10397, 17648]","{'SNP(R)': {'pre': 387, 'post': 1777, 'downstr...",NaN,"['AOTU(R)', 'AVLP(R)', 'EPA(R)', 'ICL(R)', 'IN...","['AOTU(R)', 'AVLP(R)', 'EPA(R)', 'ICL(R)', 'IN...",pC1e


In [24]:
h = z[z['0.5']==4]
x=h[h['0.75']==5]
x[x['1.0']==61]

,key,0.0,0.05,0.1,0.5,0.75,1.0,instance,celltype,pre,...,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois,type_group
id,,,,,,,,,,,,,,,,,,,,,


In [25]:
HB_node_df[HB_node_df['celltype']=='SMP550']

,key,0.0,0.05,0.1,0.5,0.75,1.0,instance,celltype,pre,...,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois,type_group
id,,,,,,,,,,,,,,,,,,,,,
452689494,789,6,32,5,234,437,498,SMP550_R,SMP550,1414,...,False,Roughly traced,PDM29,347.0,"[19511, 18367, 6016]","{'SNP(R)': {'pre': 1248, 'post': 3275, 'downst...",pSP7 candidates,"['AVLP(R)', 'INP', 'LH(R)', 'PLP(R)', 'SCL(R)'...","['AVLP(R)', 'INP', 'LH(R)', 'PLP(R)', 'SCL(R)'...",SMP550
579700707,1306,4,5,5,335,478,551,SMP550(PDM29)_L,SMP550,992,...,True,Leaves,NaN,NaN,NaN,"{'SNP(L)': {'pre': 981, 'post': 1089, 'downstr...",pSP7 candidates,"['SMP(L)', 'SMP(R)', 'SNP(L)', 'SNP(R)']","['SMP(L)', 'SMP(R)', 'SNP(L)', 'SNP(R)']",SMP550


In [16]:
# Pulled from Prof G's code on github (https://github.com/Gutierrez-lab/oviIN-analyses-gabrielle/blob/main/modular_sandbox.ipynb)
def modularity_merge(df1,df2,suf1,suf2):
    """Given two modularity dataframes, merge them along shared body IDs. Pass in suffixes for the columns as strings."""
    merged_mod_df = df1.merge(df2, left_on='id', right_on='id', suffixes=[suf1, suf2])
    #merged_mod_df = df1.merge(df2, left_on='id', right_on='id', suffixes=['_oviHB', '_wholeHB'])
    return merged_mod_df

# Defining path to whole brain data
preproc_dir = "hemibrain/preprocessed-" + "v1.2"
preproc_nodes = "preprocessed_nodes.csv"

# Creating dataframe
WB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_nodes), index_col=0)
WB_node_df["type_group"] = HB_node_df["celltype"]

In [17]:
# Defining base res and suffixes
res = '0.0'
df1_suf = '_oviHB'
df2_suf = '_wholeHB'

# Using merge function from code cell above
mod_merge_df = modularity_merge(HB_node_df[[res]],WB_node_df[[res]],df1_suf,df2_suf)
mod_merge_df

,0.0_oviHB,0.0_wholeHB
id,,
1003215282,1,6
1005952640,2,1
1006928515,1,1
1007260806,2,1
1008024276,3,2
...,...,...
987117151,2,1
987273073,3,3
988567837,2,2


In [18]:
# defining colors for oviHB clusters
import bokeh.palettes
cmap = bokeh.palettes.tol['Bright'][5]

#create dictionary
color_dict = dict(zip(mod_merge_df[res + df1_suf].unique(), cmap))
color_dict

{1: '#4477AA', 2: '#EE6677', 3: '#228833', 4: '#CCBB44', 5: '#66CCEE'}

In [19]:
chi1 = res + df1_suf
chi2 = res + df2_suf

jm = joint_marginal(mod_merge_df, chi1, chi2, include_fraction=True)

# map colors into the dataframe
jm['color']=jm['0.0_oviHB'].map(color_dict)

# sort the clusters on the y axis to get a more "diagonal" plot
yrange = jm.sort_values([f"{chi2}_fraction"], ascending=False).groupby(chi2).agg({chi1: "first", f"{chi2}_fraction": "first", "joint_count": "first"}).sort_values([chi1, "joint_count"], ascending=[True, False]).index

# make a bokeh figure
f = figure(title=f"Module at chi2 = {chi2} vs. module at chi1 = {chi1}",
x_range=FactorRange(factors=[str(i + 1) for i in range(jm[chi1].max())]),
y_range=FactorRange(factors=[str(y) for y in yrange]),
width=600, height=700)
jm["x"] = jm[chi1].apply(str)  # bokeh factor range has to have strings, so we have to convert these
jm["y"] = jm[chi2].apply(str)

f.rect(x="x", y="y", width=f"{chi1}_fraction", height=f"{chi2}_fraction", source=jm, fill_color='color', line_color='color')
f.add_tools(HoverTool(tooltips={"Neurons": "@joint_count (@joint_fraction{%%} of Hemibrain)",
                                f"Fraction of {chi2}": f"@{{{chi2}_fraction}}{{2.%%}}",
                                f"Fraction of {chi1}": f"@{{{chi1}_fraction}}{{2.%%}}"}))
f.xaxis.axis_label = 'Module in ' +chi1
f.yaxis.axis_label = 'Module in ' +chi2

show(f)

In [20]:
jm

,0.0_oviHB,0.0_wholeHB,joint_count,0.0_oviHB_count,0.0_wholeHB_count,joint_fraction,0.0_oviHB_fraction,0.0_wholeHB_fraction,color,x,y
0,1,3,349,445,1221,0.190502,0.784270,0.285831,#4477AA,1,3
1,3,3,287,498,1221,0.156659,0.576305,0.235053,#228833,3,3
2,5,3,271,325,1221,0.147926,0.833846,0.221949,#66CCEE,5,3
3,2,3,229,470,1221,0.125000,0.487234,0.187551,#EE6677,2,3
7,4,3,85,94,1221,0.046397,0.904255,0.069615,#CCBB44,4,3
6,1,1,89,445,279,0.048581,0.200000,0.318996,#4477AA,1,1
19,3,1,2,498,279,0.001092,0.004016,0.007168,#228833,3,1
22,5,1,1,325,279,0.000546,0.003077,0.003584,#66CCEE,5,1
5,2,1,180,470,279,0.098253,0.382979,0.645161,#EE6677,2,1
12,4,1,7,94,279,0.003821,0.074468,0.025090,#CCBB44,4,1


In [21]:
# save f as a svg file
from bokeh.io import export_svgs

# option 1
f.output_backend = "svg"

# option 2
export_svgs(f, filename='manuscript_figs/Figure5B_jointmarginal.svg')

The geckodriver version (0.34.0) detected in PATH at /Users/rhessa/miniconda3/envs/flybrainlab/bin/geckodriver might not be compatible with the detected firefox version (122.0); currently, geckodriver 0.35.0 is recommended for firefox 122.*, so it is advised to delete the driver in PATH and retry


['manuscript_figs/Figure5B_jointmarginal.svg']

In [28]:
from reportlab.graphics import renderPDF
from svglib.svglib import svg2rlg

# read in svg file
drawing = svg2rlg('manuscript_figs/Figure5B_jointmarginal.svg')

# save as pdf
renderPDF.drawToFile(drawing, 'manuscript_figs/Figure5B_jointmarginal.pdf')